In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

### 미국 항공사 데이터 예제

**데이터 불러오기**

In [2]:
# SparkSession 생성
spark = (SparkSession
         .builder
         .appName("SparkSQLExampleApp")
         .getOrCreate())

23/03/24 23:49:59 WARN Utils: Your hostname, choeyunseoui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.17 instead (on interface en0)
23/03/24 23:49:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 23:50:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 데이터 불러오기
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"

df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load("departuredelays.csv", schema=schema))

df.createOrReplaceTempView("us_delay_flights_tbl")

In [4]:
df.show(5)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [5]:
# 예제1. 비행거리가 1000마일 이상인 항공편
spark.sql("""SELECT distance, origin, destination
             FROM us_delay_flights_tbl WHERE distance > 1000
             ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [6]:
# 예제1. 동일한 계산을 데이터 프레임으로도 수행할 수 있다.
(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [7]:
# 샌프란시스코(SFO)와 시카고(ORD)간 2시간 이상 지연된 항공편
spark.sql("""SELECT date, delay, origin, destination
             FROM us_delay_flights_tbl
             WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
             ORDER by delay DESC""").show(10)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|02190925| 1638|   SFO|        ORD|
|01031755|  396|   SFO|        ORD|
|01022330|  326|   SFO|        ORD|
|01051205|  320|   SFO|        ORD|
|01190925|  297|   SFO|        ORD|
|02171115|  296|   SFO|        ORD|
|01071040|  279|   SFO|        ORD|
|01051550|  274|   SFO|        ORD|
|03120730|  266|   SFO|        ORD|
|01261104|  258|   SFO|        ORD|
+--------+-----+------+-----------+
only showing top 10 rows



In [8]:
# date data type 변경
from pyspark.sql.functions import *

add_df = (df.withColumn("newdate", to_timestamp(col("date"), "MMddhhmm"))
             .withColumn("month", month(col("newdate")))
             .withColumn("day", dayofmonth(col("newdate"))))

add_df.createOrReplaceTempView("us_delay_flights_tbl_add_ver")

In [9]:
add_df.show(5)

+--------+-----+--------+------+-----------+-------------------+-----+---+
|    date|delay|distance|origin|destination|            newdate|month|day|
+--------+-----+--------+------+-----------+-------------------+-----+---+
|01011245|    6|     602|   ABE|        ATL|1970-01-01 00:45:00|    1|  1|
|01020600|   -8|     369|   ABE|        DTW|1970-01-02 06:00:00|    1|  2|
|01021245|   -2|     602|   ABE|        ATL|1970-01-02 00:45:00|    1|  2|
|01020605|   -4|     602|   ABE|        ATL|1970-01-02 06:05:00|    1|  2|
|01031245|   -4|     602|   ABE|        ATL|1970-01-03 00:45:00|    1|  3|
+--------+-----+--------+------+-----------+-------------------+-----+---+
only showing top 5 rows



p.91 의 연습문제
- date 칼럼을 읽을 수 있는 포맷으로 변경하고, 가장 흔하게 지연이 발생한 날짜나 달을 찾아보기

In [10]:
spark.sql("""SELECT month, day, count(*), avg(delay)
             FROM us_delay_flights_tbl_add_ver
             WHERE ORIGIN = 'SFO' AND DESTINATION = 'ORD'
             GROUP BY month, day
             ORDER by avg(delay) DESC, month , day """).show(10)

+-----+---+--------+------------------+
|month|day|count(1)|        avg(delay)|
+-----+---+--------+------------------+
|    2| 19|      12|             138.0|
|    3| 12|      13| 67.15384615384616|
|    2| 17|      12|             51.25|
|    1|  5|      10|              44.5|
|    3| 26|      13| 43.30769230769231|
|    1|  2|      12|             39.25|
|    1| 30|      13| 36.15384615384615|
|    3|  5|      12|35.916666666666664|
|    2|  8|       8|            33.125|
|    3| 17|      13|31.692307692307693|
+-----+---+--------+------------------+
only showing top 10 rows



In [11]:
# 지연 정도를 나타내는 Flight_Delays
spark.sql("""SELECT delay, origin, destination,
             CASE
                WHEN delay > 360 THEN 'Very Long Delays'
                WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
                WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
                WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delays'
                WHEN delay = 0 THEN 'No Delays'
                ELSE 'Early'
            END AS Flight_Delays
            FROM us_delay_flights_tbl
            ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



### SQL과 테이블 뷰

In [12]:
# 데이터 베이스 생성
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

In [13]:
# 관리형 테이블 생성
flights_df = spark.read.csv("departuredelays.csv", schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

23/03/25 00:00:36 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/25 00:00:36 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/25 00:00:38 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [18]:
# 비관리형 테이블 생성
# 1. sql
spark.sql("""
    CREATE TABLE us_delay_flights_tbl (
    date STRING,
    delay INT,
    distance INT,
    origin STRING,
    destination STRING
    )
    USING csv OPTIONS (PATH 'departuredelays.csv')
    """)

DataFrame[]

In [21]:
# 2. 데이터 프레임 API
(flights_df
 .write
 .option("path", "/tmp/data/us_flights_delay")
 .saveAsTable("us_delay_flights_tbl2"))

23/03/25 00:03:13 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/25 00:03:13 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers


23/03/25 00:03:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [29]:
# 항공사 데이터에서 전역/일반 임시 뷰 생성
df_sfo = spark.sql("""SELECT date, delay, origin, destination
                      FROM us_delay_flights_tbl
                      WHERE origin = 'SFO'""")
df_jfk = spark.sql("""SELECT date, delay, origin, destination
                      FROM us_delay_flights_tbl
                      WHERE origin = 'JFK'""")

# create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [33]:
# 전역 임시 뷰 접근
# 방법1
spark.read.table("global_temp.us_origin_airport_SFO_global_tmp_view").show(5)
# 방법2
spark.sql("SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01011250|   55|   SFO|        JFK|
|01012230|    0|   SFO|        JFK|
|01010705|   -7|   SFO|        JFK|
|01010620|   -3|   SFO|        MIA|
|01010915|   -3|   SFO|        LAX|
+--------+-----+------+-----------+
only showing top 5 rows

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01011250|   55|   SFO|        JFK|
|01012230|    0|   SFO|        JFK|
|01010705|   -7|   SFO|        JFK|
|01010620|   -3|   SFO|        MIA|
|01010915|   -3|   SFO|        LAX|
+--------+-----+------+-----------+
only showing top 5 rows



In [32]:
# 일반 임시 뷰 접근
# 방법1
spark.read.table("us_origin_airport_JFK_tmp_view").show(5)
# 방법2
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01010900|   14|   JFK|        LAX|
|01011200|   -3|   JFK|        LAX|
|01011900|    2|   JFK|        LAX|
|01011700|   11|   JFK|        LAS|
|01010800|   -1|   JFK|        SFO|
+--------+-----+------+-----------+
only showing top 5 rows

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01010900|   14|   JFK|        LAX|
|01011200|   -3|   JFK|        LAX|
|01011900|    2|   JFK|        LAX|
|01011700|   11|   JFK|        LAS|
|01010800|   -1|   JFK|        SFO|
+--------+-----+------+-----------+
only showing top 5 rows



In [34]:
# 뷰 드롭
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

True

In [7]:
spark.stop()